# OrthodoxAI - RAG

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

This notebook is dedicated to prototyping and testing **Retrieval-Augmented Generation (RAG)** methods within the **OrthodoxAI** framework.

It serves as a sandbox to:

- Evaluate different libraries and methods for building vector stores  
- Benchmark the performance and timing of the retrieval process  
- Isolate and debug components before integrating them into the main agent system

This notebook is focused solely on the retrieval layer of RAG, without involving agent roles or generation logic at this stage.

## Libraries